In [16]:
import imutils
import cv2 #to import cv2, the modul opencv-python needs to be installed
import os
import math
import numpy as np

In [4]:
'''
Use to load image by URL or file path
'''
def loadImage(filepath, resize_width = 0):
    
    if filepath.startswith("http"):
        image = imutils.url_to_image(filepath)
    else:
        #check if input file exists
        if not os.path.isfile(filepath):
            raise FileNotFoundError("Input file could not be read. Please check your file path.")
        image = cv2.imread(filepath)
    #check if opencv was successful reading the image
    if image is None:
        raise ValueError("Input file exists but could not be read by opencv (probably not a valid image).")
    
    height, width, _ = image.shape
    if resize_width and width > resize_width:
        scale = resize_width / width
        newHeight = height * scale
        image = cv2.resize(image,(resize_width, int(newHeight)))
    return image

# How to work with homography matrices

This notebook explains the basics of the homography mapping between a query region and a result region in the tool VISE. If you want to know, where VISE returns such a homography matrix on search, have a look in the usage guide (advanced usage) of this session first.

## Homography Matrix

A homography matrix as used in VISE is defined as a matrix of the shape

$$ 
M = \begin{matrix}
a & b & c\\
d & e & f\\
0 & 0 & 1
\end{matrix}
$$

This matrix, called a(n affine) transformation matrix, is a composition of multiple matrices, representing *rotation*, *scaling*, *shear* and *translation*.

### Rotation

The query shape for a match can be rotated in the 2d plane to get the resulting match.

The rotation angle is derived from the homography matrix by calculating $$rot = atan2(b,a)$$

### Scaling

The query shape can be enlarged or shrunken to get the resulting match.

The twodimensional scaling factor is derived from the homography matrix by calculating $$scal_x, scal_y = \frac{det(M)}{\sqrt{a^2+b^2}} = \frac{ae-bd}{\sqrt{a^2+b^2}}$$

### Shear
The query shape can be sheared to get the resulting match, thus changing the corner angles of the initial rectangle but keeping all parallel lines parallel.

Shear represents rotation on the third axis as perceived on the 2d plane.

The shear factor $s$, which determines the displacement of a point $(x,y)$ to $(x,y+sx)$ is derived from the homography matrix by calculating

$$
s =\left\{
\begin{array}{ll}
0 &\text{if }det(M)=0 \\ 
\frac{ad+be}{det(M)} &\text{otherwise}.
\end{array} 
\right.
$$

In [38]:
#for exercise, filenames have been replaced with IIIF url

viseresultJson = {
    "PNAME":"winterschool-example",
    "QUERY":{
        "file_id":93,
        "filename":"https://content.staatsbibliothek-berlin.de/dc/846205432-0034/full/,300/0/default.jpg",
        "x":25,"y":121,"width":152,"height":110
    },
    "RESULT_SIZE":31,
    "RESULT":[
        {"file_id":93,"filename":"https://content.staatsbibliothek-berlin.de/dc/846205432-0034/full/,300/0/default.jpg","score":177.016,"H":[1,0,0,0,1,0,0,0,1]},
        {"file_id":25,"filename":"https://content.staatsbibliothek-berlin.de/dc/846205432-0244/full/,300/0/default.jpg","score":29.0008,"H":[1.21121,0,-19.0093,0.100599,1.23619,-130.709,0,0,1]},
        {"file_id":840,"filename":"some_other_image.jpg","score":5.00003,"H":[1.09996,0,25.0013,-0.312663,1.01493,-39.7847,0,0,1]}
    ]
}

## Exercise: Assess Homomgraphy Results

In the above data example, due to inconvenient circumstances, we do not know how the image in the last result looks like. However, by looking at the homography matrix, we can determine how the resulting shape would look like.

Since we have applied VISE to detection of illustrations on book pages, the expected results of the homography mapping are somewhat limited. We do not expect an illustration to be freely rotated (but 90° rotation might be fine). We do not expect the aspect ratio of the result region to differ vastly from the input region (vertical / horizontal scaling). We do not expect strong shearing.


### Option 1:

Find a way do draw query rectangle and result polygon to assess the match.
An easy option might be provided by [shapely.affinity.affine_transform](https://shapely.readthedocs.io/en/latest/manual.html#affine-transformations).

### Option 2:

Assess the described operations (rotation, scaling,...) individually by calculating them for the last result in the results list (either by writing functions or by calculating by hand)




In [103]:
### INSERT CODE FOR EXERCISE ###